<a href="https://colab.research.google.com/github/Praxis-QR/RDWH/blob/main/Cross_Tabs_with_OLAP_Cube_Normalised_KeyValueTables.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![alt text](https://github.com/Praxis-QR/RDWH/raw/main/images/YantraJaalBanner.png)<br>


<hr>

[Prithwis Mukerjee](http://www.linkedin.com/in/prithwis)<br>

#Generating Cross Tabs
Companion codes for [this slide deck](https://docs.google.com/presentation/d/1wQzZoHFo3NVk-dMTebHA4LH7B4KYmOfiSammjsSRGL0/edit?usp=sharing)

In [1]:
from datetime import datetime
import pytz
print('ॐ श्री सरस्वत्यै नमः',datetime.now(pytz.timezone('Asia/Calcutta')))
!python --version
!lsb_release -a

ॐ श्री सरस्वत्यै नमः 2023-09-04 10:24:20.193130+05:30
Python 3.10.12
No LSB modules are available.
Distributor ID:	Ubuntu
Description:	Ubuntu 22.04.2 LTS
Release:	22.04
Codename:	jammy


#Install MySQL

In [ ]:
# Some system parameters
#!cat /proc/cpuinfo
#!cat /proc/meminfo

In [2]:
!apt-get update > /dev/null
!apt-get -y install mysql-server > /dev/null
!/etc/init.d/mysql restart
!mysql -e 'create database praxisDB'
!mysql -e 'show databases'

 * Stopping MySQL database server mysqld
   ...done.
 * Starting MySQL database server mysqld
su: warning: cannot change directory to /nonexistent: No such file or directory
   ...done.
+--------------------+
| Database           |
+--------------------+
| information_schema |
| mysql              |
| performance_schema |
| praxisDB           |
| sys                |
+--------------------+


#Create, Load Normalised Fact Table

In [3]:
!mysql praxisDB -e 'DROP table if exists CarSalesFact_N'
!mysql praxisDB -e 'CREATE TABLE if not exists CarSalesFact_N ( Txn int(2) ,  Dim  varchar(8) ,  DimValue varchar(8) ) '
!mysql praxisDB -e 'desc CarSalesFact_N'

+----------+------------+------+-----+---------+-------+
| Field    | Type       | Null | Key | Default | Extra |
+----------+------------+------+-----+---------+-------+
| Txn      | int        | YES  |     | NULL    |       |
| Dim      | varchar(8) | YES  |     | NULL    |       |
| DimValue | varchar(8) | YES  |     | NULL    |       |
+----------+------------+------+-----+---------+-------+


In [4]:
!mysql praxisDB -e " \
INSERT INTO CarSalesFact_N (Txn, Dim, DimValue) VALUES \
 (1, 'Make', 'Honda'),(1, 'Type', 'SUV'),(1, 'Colour', 'Red'),\
 (2, 'Make', 'Honda'),(2, 'Type', 'SUV'),(2, 'Colour', 'Black'), \
 (3, 'Make', 'Honda'),(3, 'Type', 'Sedan'),(3, 'Colour', 'Brown'),\
 (4, 'Make', 'Toyota'),(4, 'Type', 'Compact'),(4, 'Colour', 'Red'),\
 (5, 'Make', 'Toyota'),(5, 'Colour', 'Black'),(5, 'Type', 'Sedan'),\
 (6, 'Make', 'Toyota'),(6, 'Colour', 'Black'),(6, 'Type', 'Sedan'),\
 (7, 'Make', 'Honda'),(7, 'Type', 'SUV'),(7, 'Colour', 'Black'),\
 (8, 'Make', 'Honda'),(8, 'Type', 'SUV'),(8, 'Colour', 'Red'),\
 (9, 'Make', 'Honda'),(9, 'Type', 'SUV'),(9, 'Colour', 'Black'),\
 (10, 'Make', 'Honda'),(10, 'Type', 'SUV'),(10, 'Colour', 'Black'),\
 (11, 'Make', 'Mazda'),(11, 'Type', 'SUV'),(11, 'Colour', 'Black'),\
 (12, 'Make', 'Mazda'),(12, 'Type', 'Compact'),(12, 'Colour', 'Red'),\
 (13, 'Make', 'Mazda'),(13, 'Type', 'Sedan'),(13, 'Colour', 'Brown'),\
 (14, 'Make', 'Mazda'),(14, 'Type', 'SUV'),(14, 'Colour', 'Red');\
 "


In [5]:
!mysql praxisDB -e 'select * from CarSalesFact_N'

+------+--------+----------+
| Txn  | Dim    | DimValue |
+------+--------+----------+
|    1 | Make   | Honda    |
|    1 | Type   | SUV      |
|    1 | Colour | Red      |
|    2 | Make   | Honda    |
|    2 | Type   | SUV      |
|    2 | Colour | Black    |
|    3 | Make   | Honda    |
|    3 | Type   | Sedan    |
|    3 | Colour | Brown    |
|    4 | Make   | Toyota   |
|    4 | Type   | Compact  |
|    4 | Colour | Red      |
|    5 | Make   | Toyota   |
|    5 | Colour | Black    |
|    5 | Type   | Sedan    |
|    6 | Make   | Toyota   |
|    6 | Colour | Black    |
|    6 | Type   | Sedan    |
|    7 | Make   | Honda    |
|    7 | Type   | SUV      |
|    7 | Colour | Black    |
|    8 | Make   | Honda    |
|    8 | Type   | SUV      |
|    8 | Colour | Red      |
|    9 | Make   | Honda    |
|    9 | Type   | SUV      |
|    9 | Colour | Black    |
|   10 | Make   | Honda    |
|   10 | Type   | SUV      |
|   10 | Colour | Black    |
|   11 | Make   | Mazda    |
|   11 | Type 

#Create a Standard Fact Table

##Pivot - Rows to Columns

In [6]:
!mysql praxisDB -e "\
SELECT \
  Txn,\
  GROUP_CONCAT(if(Dim = 'Make', DimValue, NULL)) AS 'Make',\
  GROUP_CONCAT(if(Dim = 'Type', DimValue, NULL)) AS 'Type',\
  GROUP_CONCAT(if(Dim = 'Colour', DimValue, NULL)) AS 'Colour'\
FROM CarSalesFact_N \
GROUP BY txn \
"

+------+--------+---------+--------+
| Txn  | Make   | Type    | Colour |
+------+--------+---------+--------+
|    1 | Honda  | SUV     | Red    |
|    2 | Honda  | SUV     | Black  |
|    3 | Honda  | Sedan   | Brown  |
|    4 | Toyota | Compact | Red    |
|    5 | Toyota | Sedan   | Black  |
|    6 | Toyota | Sedan   | Black  |
|    7 | Honda  | SUV     | Black  |
|    8 | Honda  | SUV     | Red    |
|    9 | Honda  | SUV     | Black  |
|   10 | Honda  | SUV     | Black  |
|   11 | Mazda  | SUV     | Black  |
|   12 | Mazda  | Compact | Red    |
|   13 | Mazda  | Sedan   | Brown  |
|   14 | Mazda  | SUV     | Red    |
+------+--------+---------+--------+


##Create CUBE

In [7]:
!mysql praxisDB -e "\
select csf0.Make, csf0.Type, csf0.Colour, count(txn) Units from \
(SELECT \
  Txn,\
  GROUP_CONCAT(if(Dim = 'Make', DimValue, NULL)) AS 'Make',\
  GROUP_CONCAT(if(Dim = 'Type', DimValue, NULL)) AS 'Type',\
  GROUP_CONCAT(if(Dim = 'Colour', DimValue, NULL)) AS 'Colour'\
FROM CarSalesFact_N \
GROUP BY txn \
) as csf0 \
group by \
csf0.Make, csf0.Type, csf0.Colour \
"

+--------+---------+--------+-------+
| Make   | Type    | Colour | Units |
+--------+---------+--------+-------+
| Honda  | SUV     | Red    |     2 |
| Honda  | SUV     | Black  |     4 |
| Honda  | Sedan   | Brown  |     1 |
| Toyota | Compact | Red    |     1 |
| Toyota | Sedan   | Black  |     2 |
| Mazda  | SUV     | Black  |     1 |
| Mazda  | Compact | Red    |     1 |
| Mazda  | Sedan   | Brown  |     1 |
| Mazda  | SUV     | Red    |     1 |
+--------+---------+--------+-------+


In [8]:
!mysql praxisDB -e "drop table if exists CarSalesFact_S"
!mysql praxisDB -e " \
create table CarSalesFact_S as ( \
    select csf0.Make, csf0.Type, csf0.Colour, count(txn) Units from \
        (SELECT \
        Txn,\
        GROUP_CONCAT(if(Dim = 'Make', DimValue, NULL)) AS 'Make',\
        GROUP_CONCAT(if(Dim = 'Type', DimValue, NULL)) AS 'Type',\
        GROUP_CONCAT(if(Dim = 'Colour', DimValue, NULL)) AS 'Colour'\
        FROM CarSalesFact_N \
        GROUP BY txn \
        ) as csf0 \
        group by \
        csf0.Make, csf0.Type, csf0.Colour \
) \
"
!mysql praxisDB -e "select * from CarSalesFact_S"

+--------+---------+--------+-------+
| Make   | Type    | Colour | Units |
+--------+---------+--------+-------+
| Honda  | SUV     | Red    |     2 |
| Honda  | SUV     | Black  |     4 |
| Honda  | Sedan   | Brown  |     1 |
| Toyota | Compact | Red    |     1 |
| Toyota | Sedan   | Black  |     2 |
| Mazda  | SUV     | Black  |     1 |
| Mazda  | Compact | Red    |     1 |
| Mazda  | Sedan   | Brown  |     1 |
| Mazda  | SUV     | Red    |     1 |
+--------+---------+--------+-------+


#Cross Tabs

##Level 1

In [9]:
!mysql praxisDB -e " \
  select Make, Type, Sum(Units) Count \
  from CarSalesFact_S \
  group by Make, Type \
"

+--------+---------+-------+
| Make   | Type    | Count |
+--------+---------+-------+
| Honda  | SUV     |     6 |
| Honda  | Sedan   |     1 |
| Toyota | Compact |     1 |
| Toyota | Sedan   |     2 |
| Mazda  | SUV     |     2 |
| Mazda  | Compact |     1 |
| Mazda  | Sedan   |     1 |
+--------+---------+-------+


In [10]:
!mysql praxisDB -e " \
  select Make,  Sum(Units) Count \
  from CarSalesFact_S \
  group by Make \
"

+--------+-------+
| Make   | Count |
+--------+-------+
| Honda  |     7 |
| Toyota |     3 |
| Mazda  |     4 |
+--------+-------+


In [11]:
!mysql praxisDB -e "\
    select Make, \
    sum(if(Type = 'Sedan', Units,0)) as 'Sedan', \
    sum(if(Type = 'SUV', Units, 0)) as 'SUV', \
    sum(if(Type = 'Compact', Units, 0)) as 'Compact' \
    from CarSalesFact_S \
    group by Make"

+--------+-------+------+---------+
| Make   | Sedan | SUV  | Compact |
+--------+-------+------+---------+
| Honda  |     1 |    6 |       0 |
| Toyota |     2 |    0 |       1 |
| Mazda  |     1 |    2 |       1 |
+--------+-------+------+---------+


##Level 2 - Rollup

In [12]:
!mysql praxisDB -e " \
  select Make, Type, Sum(Units) Count \
  from CarSalesFact_S \
  group by Make, Type  \
  with ROLLUP; \
"

+--------+---------+-------+
| Make   | Type    | Count |
+--------+---------+-------+
| Honda  | Sedan   |     1 |
| Honda  | SUV     |     6 |
| Honda  | NULL    |     7 |
| Mazda  | Compact |     1 |
| Mazda  | Sedan   |     1 |
| Mazda  | SUV     |     2 |
| Mazda  | NULL    |     4 |
| Toyota | Compact |     1 |
| Toyota | Sedan   |     2 |
| Toyota | NULL    |     3 |
| NULL   | NULL    |    14 |
+--------+---------+-------+


In [13]:
!mysql praxisDB -e " \
  select IFNULL(Make,'-total-') 'Make', IFNULL(Type,'-total-') 'Type', Sum(Units) Count \
  from CarSalesFact_S \
  group by Make, Type  \
  with ROLLUP; \
"

+---------+---------+-------+
| Make    | Type    | Count |
+---------+---------+-------+
| Honda   | Sedan   |     1 |
| Honda   | SUV     |     6 |
| Honda   | -total- |     7 |
| Mazda   | Compact |     1 |
| Mazda   | Sedan   |     1 |
| Mazda   | SUV     |     2 |
| Mazda   | -total- |     4 |
| Toyota  | Compact |     1 |
| Toyota  | Sedan   |     2 |
| Toyota  | -total- |     3 |
| -total- | -total- |    14 |
+---------+---------+-------+


In [14]:
!mysql praxisDB -e " \
  select Make, Sum(Units) Count \
  from CarSalesFact_S \
  group by Make  \
  with ROLLUP; \
"

+--------+-------+
| Make   | Count |
+--------+-------+
| Honda  |     7 |
| Mazda  |     4 |
| Toyota |     3 |
| NULL   |    14 |
+--------+-------+


### Roll UP even more

In [15]:
!mysql praxisDB -e " \
  select IFNULL(Make,'-total-') 'Make', Sum(Units) Count \
  from CarSalesFact_S \
  group by Make  \
  with ROLLUP; \
"

+---------+-------+
| Make    | Count |
+---------+-------+
| Honda   |     7 |
| Mazda   |     4 |
| Toyota  |     3 |
| -total- |    14 |
+---------+-------+


In [16]:
!mysql praxisDB -e " \
  select Sum(Units) 'Total' \
  from CarSalesFact_S \
"

+-------+
| Total |
+-------+
|    14 |
+-------+


In [17]:
!mysql praxisDB -e " \
  select IFNULL(Type,'-total-') 'Type', Sum(Units) Count \
  from CarSalesFact_S \
  group by Type  \
  with ROLLUP; \
"

+---------+-------+
| Type    | Count |
+---------+-------+
| Compact |     2 |
| Sedan   |     4 |
| SUV     |     8 |
| -total- |    14 |
+---------+-------+


##Level 3  - THREE crosstabs

In [18]:
!mysql praxisDB -e  "\
SELECT \
	Make,\
	SUM(IF(Type='SUV',Units,0))  'SUV',\
	SUM(IF(Type='Sedan',Units,0))  'Sedan',\
	SUM(IF(Type='Compact',Units,0))  'Compact'\
  FROM CarSalesFact_S\
  GROUP BY Make WITH ROLLUP\
"

+--------+------+-------+---------+
| Make   | SUV  | Sedan | Compact |
+--------+------+-------+---------+
| Honda  |    6 |     1 |       0 |
| Mazda  |    2 |     1 |       1 |
| Toyota |    0 |     2 |       1 |
| NULL   |    8 |     4 |       2 |
+--------+------+-------+---------+


In [19]:
!mysql praxisDB -e  "\
SELECT \
IFNULL(Make,'Totals') Dims,\
sums.SUV, sums.Sedan, sums.Compact,\
sums.SUV + sums.Sedan + sums.Compact  Totals \
FROM ( \
  SELECT \
	Make,\
	SUM(IF(Type='SUV',Units,0))  'SUV',\
	SUM(IF(Type='Sedan',Units,0))  'Sedan',\
	SUM(IF(Type='Compact',Units,0))  'Compact'\
  FROM CarSalesFact_S\
  GROUP BY Make WITH ROLLUP\
) AS sums \
"

+--------+------+-------+---------+--------+
| Dims   | SUV  | Sedan | Compact | Totals |
+--------+------+-------+---------+--------+
| Honda  |    6 |     1 |       0 |      7 |
| Mazda  |    2 |     1 |       1 |      4 |
| Toyota |    0 |     2 |       1 |      3 |
| Totals |    8 |     4 |       2 |     14 |
+--------+------+-------+---------+--------+


In [20]:
!mysql praxisDB -e  "\
SELECT \
IFNULL(Make,'Totals') Dims,\
sums.Brown, sums.Black, sums.Red,\
sums.Brown + sums.Black + sums.Red  Totals \
FROM ( \
  SELECT \
	Make,\
	SUM(IF(Colour='Brown',Units,0))  'Brown',\
	SUM(IF(Colour='Black',Units,0))  'Black',\
	SUM(IF(Colour='Red',UNits,0))  'Red'\
  FROM CarSalesFact_S\
  GROUP BY Make WITH ROLLUP\
) AS sums \
"

+--------+-------+-------+------+--------+
| Dims   | Brown | Black | Red  | Totals |
+--------+-------+-------+------+--------+
| Honda  |     1 |     4 |    2 |      7 |
| Mazda  |     1 |     1 |    2 |      4 |
| Toyota |     0 |     2 |    1 |      3 |
| Totals |     2 |     7 |    5 |     14 |
+--------+-------+-------+------+--------+


In [21]:
!mysql praxisDB -e  "\
SELECT \
IFNULL(Type,'Totals') Dims,\
sums.Brown, sums.Black, sums.Red,\
sums.Brown + sums.Black + sums.Red  Totals \
FROM ( \
  SELECT \
	Type,\
	SUM(IF(Colour='Brown',Units,0))  'Brown',\
	SUM(IF(Colour='Black',Units,0))  'Black',\
	SUM(IF(Colour='Red',UNits,0))  'Red'\
  FROM CarSalesFact_S\
  GROUP BY Type WITH ROLLUP\
) AS sums \
"

+---------+-------+-------+------+--------+
| Dims    | Brown | Black | Red  | Totals |
+---------+-------+-------+------+--------+
| Compact |     0 |     0 |    2 |      2 |
| Sedan   |     2 |     2 |    0 |      4 |
| SUV     |     0 |     5 |    3 |      8 |
| Totals  |     2 |     7 |    5 |     14 |
+---------+-------+-------+------+--------+


##Level 3 A - the UNION ALL / CUBE

In [22]:
!mysql praxisDB -e  "\
SELECT Make, Type, SUM(Units) \
FROM CarSalesFact_S \
GROUP BY Make, Type \
UNION ALL \
SELECT Make, 'ALL', SUM(Units) \
FROM CarSalesFact_S \
GROUP BY Make \
UNION ALL \
SELECT 'ALL', Type, SUM(Units) \
FROM CarSalesFact_S \
GROUP BY Type \
UNION ALL \
SELECT 'ALL', 'ALL', SUM(Units) \
FROM CarSalesFact_S\
"



+--------+---------+------------+
| Make   | Type    | SUM(Units) |
+--------+---------+------------+
| Honda  | SUV     |          6 |
| Honda  | Sedan   |          1 |
| Toyota | Compact |          1 |
| Toyota | Sedan   |          2 |
| Mazda  | SUV     |          2 |
| Mazda  | Compact |          1 |
| Mazda  | Sedan   |          1 |
| Honda  | ALL     |          7 |
| Toyota | ALL     |          3 |
| Mazda  | ALL     |          4 |
| ALL    | SUV     |          8 |
| ALL    | Sedan   |          4 |
| ALL    | Compact |          2 |
| ALL    | ALL     |         14 |
+--------+---------+------------+


#Insert an additional fact in the Fact table

In [23]:
!mysql praxisDB -e "\
INSERT INTO CarSalesFact_N (Txn, Dim, DimValue) VALUES \
 (1, 'Price', '5000'),\
 (2, 'Price', '6000'),\
 (3, 'Price', '5000'),\
 (4, 'Price', '7000'),\
 (5, 'Price', '5000'),\
 (6, 'Price', '4000'),\
 (7, 'Price', '3000'),\
 (8, 'Price', '2000'),\
 (9, 'Price', '4000'),\
 (10, 'Price', '5000'),\
 (11, 'Price', '4000'),\
 (12, 'Price', '5000'),\
 (13, 'Price', '4000'),\
 (14, 'Price', '3000');\
 "

In [24]:
!mysql praxisDB -e 'select * from CarSalesFact_N'

+------+--------+----------+
| Txn  | Dim    | DimValue |
+------+--------+----------+
|    1 | Make   | Honda    |
|    1 | Type   | SUV      |
|    1 | Colour | Red      |
|    2 | Make   | Honda    |
|    2 | Type   | SUV      |
|    2 | Colour | Black    |
|    3 | Make   | Honda    |
|    3 | Type   | Sedan    |
|    3 | Colour | Brown    |
|    4 | Make   | Toyota   |
|    4 | Type   | Compact  |
|    4 | Colour | Red      |
|    5 | Make   | Toyota   |
|    5 | Colour | Black    |
|    5 | Type   | Sedan    |
|    6 | Make   | Toyota   |
|    6 | Colour | Black    |
|    6 | Type   | Sedan    |
|    7 | Make   | Honda    |
|    7 | Type   | SUV      |
|    7 | Colour | Black    |
|    8 | Make   | Honda    |
|    8 | Type   | SUV      |
|    8 | Colour | Red      |
|    9 | Make   | Honda    |
|    9 | Type   | SUV      |
|    9 | Colour | Black    |
|   10 | Make   | Honda    |
|   10 | Type   | SUV      |
|   10 | Colour | Black    |
|   11 | Make   | Mazda    |
|   11 | Type 

#Create a 'standard' CUBE Fact table
What is [GROUP_CONCAT](https://www.google.co.in/search?client=opera&q=group_concat+in+mysql&sourceid=opera&ie=UTF-8&oe=UTF-8)?

In [25]:
!mysql praxisDB -e "\
SELECT \
  Txn,\
  GROUP_CONCAT(if(Dim = 'Make', DimValue, NULL)) AS 'Make',\
  GROUP_CONCAT(if(Dim = 'Type', DimValue, NULL)) AS 'Type',\
  GROUP_CONCAT(if(Dim = 'Colour', DimValue, NULL)) AS 'Colour',\
  GROUP_CONCAT(if(Dim = 'Price', DimValue, NULL)) AS 'Price'\
FROM CarSalesFact_N \
GROUP BY txn\
"

+------+--------+---------+--------+-------+
| Txn  | Make   | Type    | Colour | Price |
+------+--------+---------+--------+-------+
|    1 | Honda  | SUV     | Red    | 5000  |
|    2 | Honda  | SUV     | Black  | 6000  |
|    3 | Honda  | Sedan   | Brown  | 5000  |
|    4 | Toyota | Compact | Red    | 7000  |
|    5 | Toyota | Sedan   | Black  | 5000  |
|    6 | Toyota | Sedan   | Black  | 4000  |
|    7 | Honda  | SUV     | Black  | 3000  |
|    8 | Honda  | SUV     | Red    | 2000  |
|    9 | Honda  | SUV     | Black  | 4000  |
|   10 | Honda  | SUV     | Black  | 5000  |
|   11 | Mazda  | SUV     | Black  | 4000  |
|   12 | Mazda  | Compact | Red    | 5000  |
|   13 | Mazda  | Sedan   | Brown  | 4000  |
|   14 | Mazda  | SUV     | Red    | 3000  |
+------+--------+---------+--------+-------+


In [26]:
!mysql praxisDB -e "\
SELECT \
  CSF2.Make, CSF2.Type, CSF2.Colour,CSF2.Price \
FROM \
(SELECT \
  Txn,\
  GROUP_CONCAT(if(Dim = 'Make', DimValue, NULL)) AS 'Make',\
  GROUP_CONCAT(if(Dim = 'Type', DimValue, NULL)) AS 'Type',\
  GROUP_CONCAT(if(Dim = 'Colour', DimValue, NULL)) AS 'Colour',\
  GROUP_CONCAT(if(Dim = 'Price', DimValue, NULL)) AS 'Price'\
FROM CarSalesFact_N \
GROUP BY txn \
 ) as CSF2\
"


+--------+---------+--------+-------+
| Make   | Type    | Colour | Price |
+--------+---------+--------+-------+
| Honda  | SUV     | Red    | 5000  |
| Honda  | SUV     | Black  | 6000  |
| Honda  | Sedan   | Brown  | 5000  |
| Toyota | Compact | Red    | 7000  |
| Toyota | Sedan   | Black  | 5000  |
| Toyota | Sedan   | Black  | 4000  |
| Honda  | SUV     | Black  | 3000  |
| Honda  | SUV     | Red    | 2000  |
| Honda  | SUV     | Black  | 4000  |
| Honda  | SUV     | Black  | 5000  |
| Mazda  | SUV     | Black  | 4000  |
| Mazda  | Compact | Red    | 5000  |
| Mazda  | Sedan   | Brown  | 4000  |
| Mazda  | SUV     | Red    | 3000  |
+--------+---------+--------+-------+


In [27]:
!mysql praxisDB -e "\
SELECT \
  CSF2.Make, CSF2.Type, CSF2.Colour, sum(CSF2.Price) Value, count(txn) Units \
FROM \
(SELECT \
  Txn,\
  GROUP_CONCAT(if(Dim = 'Make', DimValue, NULL)) AS 'Make',\
  GROUP_CONCAT(if(Dim = 'Type', DimValue, NULL)) AS 'Type',\
  GROUP_CONCAT(if(Dim = 'Colour', DimValue, NULL)) AS 'Colour',\
  GROUP_CONCAT(if(Dim = 'Price', DimValue, NULL)) AS 'Price' \
FROM CarSalesFact_N \
GROUP BY txn \
 ) as CSF2 \
group by CSF2.Make, CSF2.Type, CSF2.Colour\
"


+--------+---------+--------+-------+-------+
| Make   | Type    | Colour | Value | Units |
+--------+---------+--------+-------+-------+
| Honda  | SUV     | Red    |  7000 |     2 |
| Honda  | SUV     | Black  | 18000 |     4 |
| Honda  | Sedan   | Brown  |  5000 |     1 |
| Toyota | Compact | Red    |  7000 |     1 |
| Toyota | Sedan   | Black  |  9000 |     2 |
| Mazda  | SUV     | Black  |  4000 |     1 |
| Mazda  | Compact | Red    |  5000 |     1 |
| Mazda  | Sedan   | Brown  |  4000 |     1 |
| Mazda  | SUV     | Red    |  3000 |     1 |
+--------+---------+--------+-------+-------+


In [28]:
!mysql praxisDB -e "drop table if exists CarSalesFact_S2"
!mysql praxisDB -e " \
create table CarSalesFact_S2 as ( \
            SELECT \
        CSF2.Make, CSF2.Type, CSF2.Colour, sum(CSF2.Price) Value, count(txn) Units \
        FROM \
        (SELECT \
        Txn,\
        GROUP_CONCAT(if(Dim = 'Make', DimValue, NULL)) AS 'Make',\
        GROUP_CONCAT(if(Dim = 'Type', DimValue, NULL)) AS 'Type',\
        GROUP_CONCAT(if(Dim = 'Colour', DimValue, NULL)) AS 'Colour',\
        GROUP_CONCAT(if(Dim = 'Price', DimValue, NULL)) AS 'Price' \
        FROM CarSalesFact_N \
        GROUP BY txn \
        ) as CSF2 \
        group by CSF2.Make, CSF2.Type, CSF2.Colour\
) \
"
!mysql praxisDB -e "select * from CarSalesFact_S2"

+--------+---------+--------+-------+-------+
| Make   | Type    | Colour | Value | Units |
+--------+---------+--------+-------+-------+
| Honda  | SUV     | Red    |  7000 |     2 |
| Honda  | SUV     | Black  | 18000 |     4 |
| Honda  | Sedan   | Brown  |  5000 |     1 |
| Toyota | Compact | Red    |  7000 |     1 |
| Toyota | Sedan   | Black  |  9000 |     2 |
| Mazda  | SUV     | Black  |  4000 |     1 |
| Mazda  | Compact | Red    |  5000 |     1 |
| Mazda  | Sedan   | Brown  |  4000 |     1 |
| Mazda  | SUV     | Red    |  3000 |     1 |
+--------+---------+--------+-------+-------+


#Cross Tab

In [29]:
!mysql praxisDB -e "select * from CarSalesFact_S2"

+--------+---------+--------+-------+-------+
| Make   | Type    | Colour | Value | Units |
+--------+---------+--------+-------+-------+
| Honda  | SUV     | Red    |  7000 |     2 |
| Honda  | SUV     | Black  | 18000 |     4 |
| Honda  | Sedan   | Brown  |  5000 |     1 |
| Toyota | Compact | Red    |  7000 |     1 |
| Toyota | Sedan   | Black  |  9000 |     2 |
| Mazda  | SUV     | Black  |  4000 |     1 |
| Mazda  | Compact | Red    |  5000 |     1 |
| Mazda  | Sedan   | Brown  |  4000 |     1 |
| Mazda  | SUV     | Red    |  3000 |     1 |
+--------+---------+--------+-------+-------+


##Level 1 - Partial Cross Tab

In [30]:
!mysql praxisDB -e "\
select Make, Type, sum(Units) Count, sum(Value) Total \
from \
CarSalesFact_S2 \
group by Make, Type\
"

+--------+---------+-------+-------+
| Make   | Type    | Count | Total |
+--------+---------+-------+-------+
| Honda  | SUV     |     6 | 25000 |
| Honda  | Sedan   |     1 |  5000 |
| Toyota | Compact |     1 |  7000 |
| Toyota | Sedan   |     2 |  9000 |
| Mazda  | SUV     |     2 |  7000 |
| Mazda  | Compact |     1 |  5000 |
| Mazda  | Sedan   |     1 |  4000 |
+--------+---------+-------+-------+


##Level 2 Roll UP

In [31]:
!mysql praxisDB -e "\
select Make, Type, sum(Units) Count, sum(Value) Total \
from \
CarSalesFact_S2 \
group by Make, Type with ROllUP\
"

+--------+---------+-------+-------+
| Make   | Type    | Count | Total |
+--------+---------+-------+-------+
| Honda  | Sedan   |     1 |  5000 |
| Honda  | SUV     |     6 | 25000 |
| Honda  | NULL    |     7 | 30000 |
| Mazda  | Compact |     1 |  5000 |
| Mazda  | Sedan   |     1 |  4000 |
| Mazda  | SUV     |     2 |  7000 |
| Mazda  | NULL    |     4 | 16000 |
| Toyota | Compact |     1 |  7000 |
| Toyota | Sedan   |     2 |  9000 |
| Toyota | NULL    |     3 | 16000 |
| NULL   | NULL    |    14 | 62000 |
+--------+---------+-------+-------+


##Level 3 Full Cross Tab

In [32]:
!mysql praxisDB -e  "\
SELECT \
IFNULL(Make,'Totals') Dims,\
sums.SUV, sums.Sedan, sums.Compact,\
sums.SUV + sums.Sedan + sums.Compact  Totals \
FROM ( \
  SELECT \
	Make,\
	SUM(IF(Type='SUV',Units,0))  'SUV',\
	SUM(IF(Type='Sedan',Units,0))  'Sedan',\
	SUM(IF(Type='Compact',Units,0))  'Compact'\
  FROM CarSalesFact_S2\
  GROUP BY Make WITH ROLLUP\
) AS sums \
"

+--------+------+-------+---------+--------+
| Dims   | SUV  | Sedan | Compact | Totals |
+--------+------+-------+---------+--------+
| Honda  |    6 |     1 |       0 |      7 |
| Mazda  |    2 |     1 |       1 |      4 |
| Toyota |    0 |     2 |       1 |      3 |
| Totals |    8 |     4 |       2 |     14 |
+--------+------+-------+---------+--------+


In [33]:
!mysql praxisDB -e  "\
SELECT \
IFNULL(Make,'Totals') Dims,\
sums.SUV, sums.Sedan, sums.Compact,\
sums.SUV + sums.Sedan + sums.Compact  Totals \
FROM ( \
  SELECT \
	Make,\
	SUM(IF(Type='SUV',Value,0))  'SUV',\
	SUM(IF(Type='Sedan',Value,0))  'Sedan',\
	SUM(IF(Type='Compact',Value,0))  'Compact'\
  FROM CarSalesFact_S2\
  GROUP BY Make WITH ROLLUP\
) AS sums \
"

+--------+-------+-------+---------+--------+
| Dims   | SUV   | Sedan | Compact | Totals |
+--------+-------+-------+---------+--------+
| Honda  | 25000 |  5000 |       0 |  30000 |
| Mazda  |  7000 |  4000 |    5000 |  16000 |
| Toyota |     0 |  9000 |    7000 |  16000 |
| Totals | 32000 | 18000 |   12000 |  62000 |
+--------+-------+-------+---------+--------+


#Cross Tab - Without Intermediate Cube Table

##Simple Cross Tab

In [34]:
!mysql praxisDB --table -e "\
select Make, Type, Sum(Units) Count \
from \
(SELECT 	 \
  CSF2.Make, CSF2.Type, CSF2.Colour, sum(CSF2.Price) Value, count(txn) Units \
FROM \
(SELECT \
  Txn, \
  GROUP_CONCAT(if(Dim = 'Make', DimValue, NULL)) AS 'Make', \
  GROUP_CONCAT(if(Dim = 'Type', DimValue, NULL)) AS 'Type', \
  GROUP_CONCAT(if(Dim = 'Colour', DimValue, NULL)) AS 'Colour', \
  GROUP_CONCAT(if(Dim = 'Price', DimValue, NULL)) AS 'Price' \
FROM CarSalesFact_N \
GROUP BY txn \
 ) as CSF2 \
group by CSF2.Make, CSF2.Type, CSF2.Colour \
) as CSF3 \
group by Make, Type \
"

+--------+---------+-------+
| Make   | Type    | Count |
+--------+---------+-------+
| Honda  | SUV     |     6 |
| Honda  | Sedan   |     1 |
| Toyota | Compact |     1 |
| Toyota | Sedan   |     2 |
| Mazda  | SUV     |     2 |
| Mazda  | Compact |     1 |
| Mazda  | Sedan   |     1 |
+--------+---------+-------+


##Better Cross Tab

In [35]:
%%writefile query.sql
-- ------------------------------------------------------------
select Make, Type, Sum(Units) Count
from
(SELECT
  CSF2.Make, CSF2.Type, CSF2.Colour, sum(CSF2.Price) Value, count(txn) Units
FROM
(SELECT
  Txn,
  GROUP_CONCAT(if(Dim = 'Make', DimValue, NULL)) AS 'Make',
  GROUP_CONCAT(if(Dim = 'Type', DimValue, NULL)) AS 'Type',
  GROUP_CONCAT(if(Dim = 'Colour', DimValue, NULL)) AS 'Colour',
  GROUP_CONCAT(if(Dim = 'Price', DimValue, NULL)) AS 'Price'
FROM CarSalesFact_N
GROUP BY txn
 ) as CSF2
group by CSF2.Make, CSF2.Type, CSF2.Colour
) as CSF3
group by Make, Type
with ROLLUP

Writing query.sql


In [36]:
!mysql praxisDB --table < query.sql

+--------+---------+-------+
| Make   | Type    | Count |
+--------+---------+-------+
| Honda  | Sedan   |     1 |
| Honda  | SUV     |     6 |
| Honda  | NULL    |     7 |
| Mazda  | Compact |     1 |
| Mazda  | Sedan   |     1 |
| Mazda  | SUV     |     2 |
| Mazda  | NULL    |     4 |
| Toyota | Compact |     1 |
| Toyota | Sedan   |     2 |
| Toyota | NULL    |     3 |
| NULL   | NULL    |    14 |
+--------+---------+-------+


##Full Cross Tab

In [37]:
%%writefile query.sql
-- ------------------------------------------------------------
SELECT
  IFNULL(Make,'Col Total Units') AS Dims,
  CSF4.SUV, CSF4.Sedan, CSF4.Compact,
  CSF4.SUV + CSF4.Sedan + CSF4.Compact AS 'Row Total Units'
FROM (
  SELECT
	Make,
	SUM(IF(Type='SUV',Units,0)) As 'SUV',
	SUM(IF(Type='Sedan',Units,0)) As 'Sedan',
	SUM(IF(Type='Compact',Units,0)) As 'Compact'
  FROM

    (SELECT
      CSF2.Make, CSF2.Type, CSF2.Colour, sum(CSF2.Price) Value, count(txn) Units
    FROM
        (SELECT
          Txn,
          GROUP_CONCAT(if(Dim = 'Make', DimValue, NULL)) AS 'Make',
          GROUP_CONCAT(if(Dim = 'Type', DimValue, NULL)) AS 'Type',
          GROUP_CONCAT(if(Dim = 'Colour', DimValue, NULL)) AS 'Colour',
          GROUP_CONCAT(if(Dim = 'Price', DimValue, NULL)) AS 'Price'
        FROM CarSalesFact_N
        GROUP BY txn
         ) as CSF2
    group by CSF2.Make, CSF2.Type, CSF2.Colour
    ) as CSF3

GROUP BY Make WITH ROLLUP
) AS CSF4



Overwriting query.sql


In [38]:
!mysql praxisDB --table < query.sql

+-----------------+------+-------+---------+-----------------+
| Dims            | SUV  | Sedan | Compact | Row Total Units |
+-----------------+------+-------+---------+-----------------+
| Honda           |    6 |     1 |       0 |               7 |
| Mazda           |    2 |     1 |       1 |               4 |
| Toyota          |    0 |     2 |       1 |               3 |
| Col Total Units |    8 |     4 |       2 |              14 |
+-----------------+------+-------+---------+-----------------+


In [39]:
%%writefile query.sql
-- ------------------------------------------------------------
SELECT
  IFNULL(Make,'Col Total Value') AS Dims,
  CSF4.SUV, CSF4.Sedan, CSF4.Compact,
  CSF4.SUV + CSF4.Sedan + CSF4.Compact AS 'Row Total Value'
FROM (
  SELECT
	Make,
	SUM(IF(Type='SUV',Value,0)) As 'SUV',
	SUM(IF(Type='Sedan',Value,0)) As 'Sedan',
	SUM(IF(Type='Compact',Value,0)) As 'Compact'
  FROM

    (SELECT
      CSF2.Make, CSF2.Type, CSF2.Colour, sum(CSF2.Price) Value, count(txn) Units
    FROM
        (SELECT
          Txn,
          GROUP_CONCAT(if(Dim = 'Make', DimValue, NULL)) AS 'Make',
          GROUP_CONCAT(if(Dim = 'Type', DimValue, NULL)) AS 'Type',
          GROUP_CONCAT(if(Dim = 'Colour', DimValue, NULL)) AS 'Colour',
          GROUP_CONCAT(if(Dim = 'Price', DimValue, NULL)) AS 'Price'
        FROM CarSalesFact_N
        GROUP BY txn
         ) as CSF2
    group by CSF2.Make, CSF2.Type, CSF2.Colour
    ) as CSF3

GROUP BY Make WITH ROLLUP
) AS CSF4


Overwriting query.sql


In [40]:
!mysql praxisDB --table < query.sql

+-----------------+-------+-------+---------+-----------------+
| Dims            | SUV   | Sedan | Compact | Row Total Value |
+-----------------+-------+-------+---------+-----------------+
| Honda           | 25000 |  5000 |       0 |           30000 |
| Mazda           |  7000 |  4000 |    5000 |           16000 |
| Toyota          |     0 |  9000 |    7000 |           16000 |
| Col Total Value | 32000 | 18000 |   12000 |           62000 |
+-----------------+-------+-------+---------+-----------------+


#Rotate the Cross Tab

In [41]:
%%writefile query.sql
-- ------------------------------------------------------------
SELECT
  IFNULL(Colour,'Col Total Value') AS Dims,
  CSF4.SUV, CSF4.Sedan, CSF4.Compact,
  CSF4.SUV + CSF4.Sedan + CSF4.Compact AS 'Row Total Value'
FROM (
  SELECT
	Colour,
	SUM(IF(Type='SUV',Value,0)) As 'SUV',
	SUM(IF(Type='Sedan',Value,0)) As 'Sedan',
	SUM(IF(Type='Compact',Value,0)) As 'Compact'
  FROM

    (SELECT
      CSF2.Make, CSF2.Type, CSF2.Colour, sum(CSF2.Price) Value, count(txn) Units
    FROM
        (SELECT
          Txn,
          GROUP_CONCAT(if(Dim = 'Make', DimValue, NULL)) AS 'Make',
          GROUP_CONCAT(if(Dim = 'Type', DimValue, NULL)) AS 'Type',
          GROUP_CONCAT(if(Dim = 'Colour', DimValue, NULL)) AS 'Colour',
          GROUP_CONCAT(if(Dim = 'Price', DimValue, NULL)) AS 'Price'
        FROM CarSalesFact_N
        GROUP BY txn
         ) as CSF2
    group by CSF2.Make, CSF2.Type, CSF2.Colour
    ) as CSF3

GROUP BY Colour WITH ROLLUP
) AS CSF4


Overwriting query.sql


In [42]:
!mysql praxisDB --table < query.sql

+-----------------+-------+-------+---------+-----------------+
| Dims            | SUV   | Sedan | Compact | Row Total Value |
+-----------------+-------+-------+---------+-----------------+
| Black           | 22000 |  9000 |       0 |           31000 |
| Brown           |     0 |  9000 |       0 |            9000 |
| Red             | 10000 |     0 |   12000 |           22000 |
| Col Total Value | 32000 | 18000 |   12000 |           62000 |
+-----------------+-------+-------+---------+-----------------+


In [43]:
%%writefile query.sql
-- ------------------------------------------------------------
SELECT
  IFNULL(Type,'Col Total Value') AS Dims,
  CSF4.Black, CSF4.Brown, CSF4.Red,
  CSF4.Black + CSF4.Brown + CSF4.Red AS 'Row Total Value'
FROM (
  SELECT
	Type,
	SUM(IF(Colour='Black',Value,0)) As 'Black',
	SUM(IF(Colour='Brown',Value,0)) As 'Brown',
	SUM(IF(Colour='Red',Value,0)) As 'Red'
  FROM

    (SELECT
      CSF2.Make, CSF2.Type, CSF2.Colour, sum(CSF2.Price) Value, count(txn) Units
    FROM
        (SELECT
          Txn,
          GROUP_CONCAT(if(Dim = 'Make', DimValue, NULL)) AS 'Make',
          GROUP_CONCAT(if(Dim = 'Type', DimValue, NULL)) AS 'Type',
          GROUP_CONCAT(if(Dim = 'Colour', DimValue, NULL)) AS 'Colour',
          GROUP_CONCAT(if(Dim = 'Price', DimValue, NULL)) AS 'Price'
        FROM CarSalesFact_N
        GROUP BY txn
         ) as CSF2
    group by CSF2.Make, CSF2.Type, CSF2.Colour
    ) as CSF3

GROUP BY Type WITH ROLLUP
) AS CSF4


Overwriting query.sql


In [44]:
!mysql praxisDB --table < query.sql

+-----------------+-------+-------+-------+-----------------+
| Dims            | Black | Brown | Red   | Row Total Value |
+-----------------+-------+-------+-------+-----------------+
| Compact         |     0 |     0 | 12000 |           12000 |
| Sedan           |  9000 |  9000 |     0 |           18000 |
| SUV             | 22000 |     0 | 10000 |           32000 |
| Col Total Value | 31000 |  9000 | 22000 |           62000 |
+-----------------+-------+-------+-------+-----------------+


In [45]:
!date

Mon Sep  4 05:01:09 AM UTC 2023


#Chronobooks <br>
![alt text](https://1.bp.blogspot.com/-lTiYBkU2qbU/X1er__fvnkI/AAAAAAAAjtE/GhDR3OEGJr4NG43fZPodrQD5kbxtnKebgCLcBGAsYHQ/s600/Footer2020-600x200.png)<hr>
Chronotantra and Chronoyantra are two science fiction novels that explore the collapse of human civilisation on Earth and then its rebirth and reincarnation both on Earth as well as on the distant worlds of Mars, Titan and Enceladus. But is it the human civilisation that is being reborn? Or is it some other sentience that is revealing itself.
If you have an interest in AI and found this material useful, you may consider buying these novels, in paperback or kindle, from [http://bit.ly/chronobooks](http://bit.ly/chronobooks)